<a href="https://colab.research.google.com/github/aytlee/sentiment140-pytorch-lstm/blob/main/sentiment140_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis

The objective of this notebook is to gain a better understanding of how to preprocess and apply a LSTM model for sentiment analysis using the Sentiment 140 dataset. This dataset was downloaded from Kaggle for ease of use, and contains 1.6M tweets that were extracted using Twitter's API. Additional information and the original dataset can be found [here](http://help.sentiment140.com/home). The original training dataset is unique in that it was automatically labeled rather than manually labeled. The notebook follows the code/tutorial from:

1. [LSTM Text Classification Using PyTorch](https://towardsdatascience.com/lstm-text-classification-using-pytorch-2c6c657f8fc0) by Raymond Chang
2. [Upgraded Sentiment Analysis](https://github.com/bentrevett/pytorch-sentiment-analysis) by Ben Trevett 


# Importing Libraries 

First, load the dataset and import all of the required libraries. While importing libraries, it was necessary to specify a version of torchtext in order to import the torchtext.legacy module. 

In [ ]:
from google.colab import files

# # Install Kaggle library
!pip install -q kaggle

# Upload kaggle API key file
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
#! kaggle competitions download -c tabular-playground-series-apr-2022
! mkdir train
! kaggle datasets download -d kazanova/sentiment140 -p /content/train/ --unzip
#! unzip tabular-playground-series-apr-2022.zip -d train

 69% 56.0M/80.9M [00:00<00:00, 108MB/s] 
100% 80.9M/80.9M [00:00<00:00, 134MB/s]


In [ ]:
# Seems to be an issue with intalling torchtext without specifying version
# cannot import torchtext.legacy module without running into issues 
!pip install torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 16.0 MB/s 
     |████████████████████████████████| 831.4 MB 2.5 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.


In [ ]:
import os 
import pandas as pd 
import numpy as np
from sklearn import model_selection 

import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import torch.autograd as autograd
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
#import torch.nn.functional as F
import torch.optim as optim 
#from torch.utils.data import Dataset, DataLoader 

import seaborn as sns 
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split

from multiprocessing import cpu_count
from sklearn.metrics import classification_report, confusion_matrix 

import plotly.graph_objects as go

In [ ]:
# TEXT = data.Field()
# LABEL = data.LabelField(dtype = torch.long)
# legacy_train, legacy_test = datasets.IMDB.splits(TEXT, LABEL)  # datasets here refers to torchtext.legacy.datasets

In [ ]:
cd train

/content/train


# Exploration of the training dataset 

According to the metadata, there are 6 columns consisting of the target, id, date, flag, user and text for the tweet. Targets are labeled 0 (negative), 2 (neutral) and 4 (positive). The training dataset only includes the negative and positive tweets, while the test dataset (found on the Stanford website) also includes the neutral tweets. 

In addition, the training dataset set only has the value "NO_QUERY" for the column flag, while the test dataset seems to have multiple values for the column. The "flag" column seems to denote some keyword that is associated with the tweet. As this notebook is meant mainly to familiarize myself with NLP using PyTorch and LSTM, I will not be using the test dataset and instead only conducting the model on a split training and validation set. 



In [ ]:
train_data = pd.read_csv(r'training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None)

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   0       1600000 non-null  int64 
 1   1       1600000 non-null  int64 
 2   2       1600000 non-null  object
 3   3       1600000 non-null  object
 4   4       1600000 non-null  object
 5   5       1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [ ]:
# Add in the column headings
train_data.columns = ['target', 'id', 'date', 'flag', 'user', 'text']

train_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# Check the proportion of the targets in the dataset
print(train_data['target'].value_counts(normalize=True))
# Apparently there are no neutral comments in the dataset, the dataset is balanced


0    0.5
4    0.5
Name: target, dtype: float64


In [ ]:
train_data['flag'].value_counts()
# No flag in this dataset


NO_QUERY    1600000
Name: flag, dtype: int64

In [ ]:
print('Number of unique users in this dataset:', train_data['user'].nunique())

Number of unique users in this dataset: 659775


In [ ]:
# Look at a sample of the negative tweets 
neg_tweets = train_data.query('target==0')[['text']].sample(10)

for i in neg_tweets.iterrows():
  print(neg_tweets.loc[i[0], 'text'], '\n')


@caguilar69 No, i work  

PS. Creeping through the grade 8s' Claremont albums on Facebook make me feel nostalgic... whatever happened to the &quot;Ant Hill Trio&quot;, lol.  

Having trouble staying asleep. It's so hot and I'm exhausted  

@OfficialJoBros u guys are going to like dis i went 2 3 or 4 stores and they are sold out of your new album!!!NOT COOL  

Is sick of having no voice!!!  

@dazzleme18 heyy i just put the final product together and it took like the last hour  lol i dunno if i can do it again  

Soooo tired...not sure I will get any writing done.  

@meerski loll I just saw this but I bet he did.  eww he needs to make it &quot;Every WOMAN&quot; 

@restartt they are expensive  

I just want to watch my Naruto Shippuuden missed episodes.  



In [ ]:
# Look at a sample of the positive tweets 
pos_tweets = train_data.query('target==4')[['text']].sample(10)

for i in pos_tweets.iterrows():
  print(pos_tweets.loc[i[0], 'text'], '\n')


@TastesLikePain do you think you'd be able to cme? We can party with Miley  

@Mr_Ant704 IDK yet but it should include a beach tho  

So its Superstar Sunday? @pradeepto is one superstar I know   #sunday 

@kingbritt I love my dentist  hope all is well wid ya  

@JonathanRKnight oh Jon u can watch me any time u like babe  xx 

NEW TOY!!!!!! Yessssss!!!! I just might sleep in my car tonight!!!  

Im inspired much by everything that is happening.    One choice, one way.    ohh .. this is just my first yime ..  plz. excuse me  tnx 

got into my apartment without keys and drunk ya!  

@rhettroberts @JamesMW78 that's a cruel &amp; tantalizing dilemma! But this is false dichotomy, I can drive with someone seated on my laps  

Happy Birthday to my son  Welcome to adulthood!! 



In [ ]:
# Relabel the positive tweets 
train_data['target'] = train_data['target'].map({4:1, 0:0})

In [ ]:
train_data['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

Based on the Stanford paper and website, the original analysis of the tweets was done automatically rather than manually reviewed. The tweets were labeled as either negative or positive based on a query term in the tweet. Their training data also used emoticons and distant supervision machine learning in order to determine the sentiment. In this notebook, we will use torchtext and a LSTM in order to process and determine the tweet sentiment. 

#Process and set up the text data for PyTorch


The cell below declares the Fields that will be used in the model. The Field specifies the data type, holds a Vocab object and determines how the data will be converted to a Tensor. I will also split the dataset using train_test_split and writing the files to a CSV file to be loaded into a TabularDataset. TabularDataset will read in columns from TSV, CSV, jSON files and convert them to a Dataset object. 

In [ ]:
TEXT = data.Field(tokenize='spacy', 
                  tokenizer_language='en_core_web_sm', 
                  include_lengths=True,
                  batch_first=True)

LABEL = data.LabelField(dtype=torch.float, batch_first=True)

fields = [('target', LABEL), ('text', TEXT)]

In [ ]:
df_train, df_valid = train_test_split(train_data[['target', 'text']], test_size=0.2, random_state=123)

In [ ]:
df_train.to_csv(r'df_train.csv', index=False)
df_valid.to_csv(r'df_valid.csv', index=False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
source_folder = os.getcwd()
trainds, validds = data.TabularDataset.splits(path=source_folder, 
                                          format='CSV',
                                          train='df_train.csv', 
                                          validation='df_valid.csv',
                                          fields=fields,
                                          skip_header=True)

In [ ]:
# Print some of the variables in the tabular dataset to see what they look like 
print(vars(trainds[0]))
print(vars(validds[-1]))



{'target': '1', 'text': ['withholding', 'information', 'am', 'I', 'Shae', '?', 'YUP', ' ']}
{'target': '1', 'text': ['Hi', ',', 'my', 'name', "'s", 'Doug', '...', 'my', 'owner', 'SQUIRREL!', '...', 'Hi', ',', 'my', 'name', "'s", 'Doug', '...']}


After creating the Dataset object, we can build a BucketIterator object using torchtext. Essentially, TabularDataset and BucketIterator are like the Dataset and DataLoader objects used in torch. However, BucketIterator will sort the data so that sequences with similar lengths are grouped together within a batch. This optimizes the batches when being input into the model and reduces the amount of padding needed. 

According to [this](https://www.analyticsvidhya.com/blog/2021/09/sentiment-analysis-with-lstm-and-torchtext-with-code-and-explanation/) article, the build_vocab function will generate the Vocab object by tokenizing the words, and provide a tensor of the token IDs for each sequence. If we take a look at the items in each batch from the BucketIterator, we see that it returns a Tensor with numerical values rather than word tokens. Although it's not done in this model, we can also pass pre-trained word embeddings to build_vocab. This is done in Ben Trevett's Upgraded Sentiment Analysis tutorial and is explained in more detail there. 

In [ ]:
TEXT.build_vocab(trainds, min_freq = 3)
LABEL.build_vocab(trainds) 

In [ ]:
# BucketIterator is an iterator that batches examples of similar lengths together 
# Minimizes the amount of padding and shuffles batches for each new epoch 

train_iter, valid_iter = data.BucketIterator.splits(
    (trainds, validds), 
    batch_size=64,
    sort_key = lambda x: len(x.text),
    sort=False,
    sort_within_batch=True,
    device=device)

In [ ]:
# Loop over bucket iterator to see what each batch and looks like 
print('PyTorchText BucketIterator\n')

for batch_no, batch in enumerate(train_iter):
    text, batch_len = batch.text
    print(text, batch_len)
    print(batch.target)
    break

  # Only look at first batch. 

PyTorchText BucketIterator

tensor([[ 517,   35,   53,  ..., 5356,   11, 2707],
        [ 434,  310, 1199,  ...,    2,    2,    2],
        [   0,  514,    8,  ..., 9988,    5,    0],
        ...,
        [   0,  767,   10,  ...,  355, 1231,    1],
        [  72,  246, 1106,  ...,    5,    0,    1],
        [2127,  252,    2,  ...,  225,    3,    1]], device='cuda:0') tensor([18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
        17, 17, 17, 17, 17, 17, 17, 17, 17, 17], device='cuda:0')
tensor([1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0.,
        1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0.,
        0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0.,
        0., 0., 0., 1., 1., 1., 0., 1., 0., 0.], device='cuda:0')


In [1]:
# In the below dictionary, you can see that stoi creates a dictionary with the token and the corresponding token ID 
# unknown and padding are 0 and 1 respectively and all other words come after 

# TEXT.vocab.stoi.items()

The model below consists of:
- an embedding layer 
- a bidirectional LSTM  
- dropout 
- fully connected layer 

The text is input into an embedding layer to create the word embeddings, then they are packed and padded before being input in the bidirectional LSTM. The last outputs from the forward and backward pass of the LSTM is then passed to the dropout layer (for regularization) before being passed to the fully connected layer. The model uses BCELoss so it's necessary to pass the outputs of the forward layer through a sigmoid function.

In [ ]:
class LSTM(nn.Module):
  def __init__(self, output_dim, hidden_dim, input_dim, embedding_dim, num_layers, dropout):
    super().__init__() # should this just be super().__init__()?

    self.embedding = nn.Embedding(input_dim, embedding_dim) 
    self.hidden_dim = hidden_dim
    self.output_dim = output_dim
    self.lstm = nn.LSTM(input_size=embedding_dim, 
                        hidden_size=hidden_dim,
                        num_layers=num_layers,
                        batch_first=True,
                        bidirectional=True)
    self.drop = nn.Dropout(dropout)

    self.fc = nn.Linear(2*hidden_dim, output_dim)

  def forward(self, text, text_len):

    text_emb = self.embedding(text)

    # Packs a Tensor containing padded sequences of variable length 
    packed_input = pack_padded_sequence(text_emb, text_len, batch_first=True, enforce_sorted=False)

    packed_output, (hidden, cell) = self.lstm(packed_input) 
    # unpack the padded sequence 
    output, output_len = pad_packed_sequence(packed_output, batch_first=True)

    # out_reduced = torch.cat((out_forward, out_reverse), 1)
    hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
    text_fea = self.drop(hidden)

    text_fea = self.fc(text_fea)
    text_fea = torch.squeeze(text_fea, 1)
    text_out = torch.sigmoid(text_fea)

    return text_out

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 128
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5 


In [ ]:
model = LSTM(OUTPUT_DIM, HIDDEN_DIM, INPUT_DIM, EMBEDDING_DIM, 
             N_LAYERS, DROPOUT).to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss() 
#criterion = criterion.to(device) # criterion needs to be sent to GPU? 
# Check if packed inputs need to be sent to GPU?

In [ ]:
def binary_accuracy(preds, y):
  '''
  Returns accuracy per batch
  '''

  rounded_preds = torch.round(preds) 
  correct = (rounded_preds == y).float() # convert to float for division
  acc = correct.sum()/len(correct)
  return acc 


A note for the train and eval functions below, the text_len variable must be a CPU tensor and not a GPU tensor like text or labels. As `train_iter` and `val_iter` were set up to be on GPU, the text and label variables will be on the GPU already. 

In [ ]:
# NOTE: The text_len variable must be a CPU tensor and not a GPU tensor like the other inputs. 

def train(mode, iterator, optimizer, criterion):
  epoch_loss = 0
  epoch_acc = 0
  model.train()

  for batch in iterator:
    labels = batch.target
    text, text_len = batch.text 
    
    text_len = text_len.cpu()

    optimizer.zero_grad()

    predictions = model(text, text_len)
    loss = criterion(predictions, labels)
    acc = binary_accuracy(predictions, labels)
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()
    epoch_acc += acc.item()

  return epoch_loss/len(iterator), epoch_acc/len(iterator)

  

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_len = batch.text
            labels = batch.target

            # labels.to(device)
            # text.to(device)
            # text_len.to(device)

            text_len = text_len.cpu()
            predictions = model(text, text_len)
            
            loss = criterion(predictions, labels)
            
            acc = binary_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5
best_valid_loss = float('inf')
train_loss_list = []
valid_loss_list = []


for epoch in range(N_EPOCHS):
  start_time = time.time()
  train_loss, train_acc = train(model, train_iter, optimizer, criterion)
  valid_loss, valid_acc = evaluate(model, valid_iter, criterion)

  end_time = time.time()
  
  train_loss_list.append(train_loss)
  valid_loss_list.append(valid_loss)

  epoch_mins, epoch_secs = epoch_time(start_time, end_time) 

  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), 'first-draft-model.pt')

  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
  print(f'\t Val.Loss: {valid_loss:.3f} | Val.Acc: {valid_acc*100:.2f}%')


Epoch: 01 | Epoch Time: 8m 11s
	Train Loss: 0.375 | Train Acc: 83.23%
	 Val.Loss: 0.337 | Val.Acc: 85.24%
Epoch: 02 | Epoch Time: 8m 10s
	Train Loss: 0.303 | Train Acc: 87.12%
	 Val.Loss: 0.330 | Val.Acc: 85.71%
Epoch: 03 | Epoch Time: 8m 11s
	Train Loss: 0.264 | Train Acc: 89.02%
	 Val.Loss: 0.336 | Val.Acc: 85.67%
Epoch: 04 | Epoch Time: 8m 10s
	Train Loss: 0.230 | Train Acc: 90.60%
	 Val.Loss: 0.351 | Val.Acc: 85.24%
Epoch: 05 | Epoch Time: 8m 9s
	Train Loss: 0.202 | Train Acc: 91.84%
	 Val.Loss: 0.377 | Val.Acc: 85.17%


After 5 epochs, the accuracy for the training dataset went from 83% to 91%; however, the validation accuracy stayed around 85%, and the accuracy does drop slightly by the last epoch. The accuracy seems to imply that the data is being overfit as it continues. Another metric that can be looked at is the classification report to see the precision and recall of the model. 

A few things that could be tried to see if the metrics can improve:
- Use pre-trained word embeddings. Glove has word embeddings for tweets specifically.
- Change the drop out value to see if increased regularization will reduce overfitting 
- Process the data more to remove certain values, e.g. usernames, links, that might not contribute to the sentiment analysis

It would also be interesting to try a simpler model, e.g. bag of words or TFIDF, to see if these models would still output similar results. Each epoch took about 8 minutes to run for this bidirectional LSTM, so if a simpler model could provide similar results, then that could lead to faster computational and processing times. 

# Determining the Last Time Step of the Backward and Forward Pass

Checked to see if `output_forward` matched `output[:, -1, :self.dimension]` or `hidden[-2, :, :]`. Found that the majority of the tensor outputs were the same but that there was one difference in certain batches with `output[:, -1, :self.dimension]`. 

out_forward: 

```
tensor([[ 0.0341, -0.0997, -0.0522,  ..., -0.0227,  0.0251,  0.3660],
        [ 0.0488,  0.0952,  0.0927,  ..., -0.1522,  0.0247,  0.2887],
        [ 0.1365,  0.0957, -0.0611,  ...,  0.0462,  0.0153, -0.2304],
        ...,
        [ 0.0477,  0.2365, -0.1074,  ...,  0.0071,  0.0043, -0.3155],
        [ 0.0067,  0.0112, -0.2509,  ..., -0.0420,  0.0087, -0.0415],
        [-0.0426,  0.3102,  0.0678,  ..., -0.0504,  0.0289, -0.3696]],
       device='cuda:0', grad_fn=<IndexBackward>)

out_forward shape: torch.Size([64, 128])
```

out_test: 
```
tensor([[ 0.0341, -0.0997, -0.0522,  ..., -0.0227,  0.0251,  0.3660],
        [ 0.0488,  0.0952,  0.0927,  ..., -0.1522,  0.0247,  0.2887],
        [ 0.1365,  0.0957, -0.0611,  ...,  0.0462,  0.0153, -0.2304],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0', grad_fn=<SliceBackward>)

out_test shape: torch.Size([64, 128])
```

hidden_forward: 

```
tensor([[ 0.0341, -0.0997, -0.0522,  ..., -0.0227,  0.0251,  0.3660],
        [ 0.0488,  0.0952,  0.0927,  ..., -0.1522,  0.0247,  0.2887],
        [ 0.1365,  0.0957, -0.0611,  ...,  0.0462,  0.0153, -0.2304],
        ...,
        [ 0.0477,  0.2365, -0.1074,  ...,  0.0071,  0.0043, -0.3155],
        [ 0.0067,  0.0112, -0.2509,  ..., -0.0420,  0.0087, -0.0415],
        [-0.0426,  0.3102,  0.0678,  ..., -0.0504,  0.0289, -0.3696]],
       device='cuda:0', grad_fn=<SliceBackward>)

hidden_forward shape: torch.Size([64, 128])
```

While `out_test`, which is `output[:, -1, :self.dimension]`, has identical values in the first part of the tensor, the latter parts are all 0's. This  does not occur in every batch. In other batches, all of the values in `out_test` are the same as both `out_forward` and `hidden_forward`. When `batch_first=True` in the LSTM model, the dimensions of output are (N, L, D*H_out), so `output[:, -1, :self.dimension]` should give the last time step of the batch up to the first hidden size dimension (forward). Unsure where this discrepancy comes from. It does seem to make more sense though, given the confusion regarding the last time step of the hidden states from output, to just take the concatenation of `hidden[-2, :, :]` and `hidden[-1, :, :]`.  

Since the output has dimensions of (batch size, sequence length, 2* size of the hidden state), it's possible that the 0s are coming from the padded values. In `out_forward`, it is only running the full length of the sequence, and nothing past that. 

From checking the output tensor, the output lengths and the text lengths, we see the following code output:

```
Output shape: torch.Size([64, 35, 256])
output_len: tensor([35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34,
        34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34,
        34, 34, 34, 34, 34, 34, 34, 34, 34, 34])
Text length minus 1: tensor([34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34,
        34, 34, 34, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33,
        33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33,
        33, 33, 33, 33, 33, 33, 33, 33, 33, 33])
```

It seems that the output shape uses the maximum text length; however, the output length for each batch shows that it's not always the same text_length for each batch. The sequences are arranged such that they are sorted by text_length in descending order. It's likely then, that the 0s are coming from the the padded values when using `output[:, -1, :self.dimension]` as this does not necessarily get the last timestep. 

Given the confusion that comes from the output tensor in bidirectional LSTMs, it seems easiest to use `hidden[-2, :, :]` and `hidden[-1, :. :]` to determine the last time steps of the forward and backward passes. Especially when the length of the sequences are not always the same.



---



---

